In [1]:
from __future__ import division
#import time
import torch 
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import cv2 
from util import *
from darknet import Darknet
from preprocess import prep_image, inp_to_image, letterbox_image
import pandas as pd
import random 
import pickle as pkl
import progressbar
import os
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

In [2]:
def get_test_input(input_dim, CUDA):
    img = cv2.imread("dog-cycle-car.png")
    img = cv2.resize(img, (input_dim, input_dim)) 
    img_ =  img[:,:,::-1].transpose((2,0,1))
    img_ = img_[np.newaxis,:,:,:]/255.0
    img_ = torch.from_numpy(img_).float()
    img_ = Variable(img_)
    
    if CUDA:
        img_ = img_.cuda()
    
    return img_

def prep_image(img, inp_dim):
    """
    Prepare image for inputting to the neural network. 
    
    Returns a Variable 
    """

    orig_im = img
    dim = orig_im.shape[1], orig_im.shape[0]
    img = (letterbox_image(orig_im, (inp_dim, inp_dim)))
    img_ = img[:,:,::-1].transpose((2,0,1)).copy()
    img_ = torch.from_numpy(img_).float().div(255.0).unsqueeze(0)
    return img_, orig_im, dim

def write(x, img):
    c1 = tuple(x[1:3].float())
    c2 = tuple(x[3:5].float())
    cls = int(x[-1])
    confidence=round(float(x[6]),2)
    label = "{0}:{1}".format(classes[cls],confidence)
    color = random.choice(colors)
    cv2.rectangle(img, c1, c2,color, 1)
    t_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_PLAIN, 1 , 1)[0]
    c2 = c1[0] + t_size[0] + 3, c1[1] + t_size[1] + 4
    cv2.rectangle(img, c1, c2,color, -1)
    cv2.putText(img, label, (c1[0], c1[1] + t_size[1] + 4), cv2.FONT_HERSHEY_PLAIN, 1, [225,255,255], 1);
    return img

def init_folder(folder):
    if not os.path.exists(folder):
        os.mkdir(folder)
    for f in os.listdir(folder):
        os.remove(folder + os.path.sep + f)
        
def get_classes(x):
    return int(x[-1])

In [3]:
#video = "test2.avi"
confidence = 0.6
nms_thresh = 0.5
cfg= os.path.join("cfg","model.cfg")
weights = "model.weights"
classes_file=os.path.join("data","model.names")
input_files='input'
reso = "256"
num_classes = 3
start=0
output_predictions='pred'
output_frames='frames'
sep=0
clipFolder='clip'

In [4]:
CUDA = torch.cuda.is_available()
bbox_attrs = 5 + num_classes
print("Loading network.....")
model = Darknet(cfg)
model.load_weights(weights)
print("Network successfully loaded")
model.net_info["height"] = reso
inp_dim = int(model.net_info["height"])
assert inp_dim % 32 == 0 
assert inp_dim > 32

if CUDA:
    model.cuda()
        
model(get_test_input(inp_dim, CUDA), CUDA)
model.eval();

Loading network.....
Network successfully loaded


/home/ds12/anaconda3/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")


In [6]:
init_folder(output_predictions)
init_folder(output_frames)
result=pd.DataFrame(columns=['file','countPerson','countBags','time','frame','timeInterval'])
ti=0
timeCursor=0
for video in os.listdir(input_files):
    video=os.path.join(input_files, video)
    cap = cv2.VideoCapture(video)
    assert cap.isOpened(), 'Cannot capture source'    
    frames = 0
    count_frames=int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print('Analyze '+video)
    bar=progressbar.ProgressBar(max_value=count_frames)
    count_files=len(os.listdir(path=output_frames))
    last_success_frame=0
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            img, orig_im, dim = prep_image(frame, inp_dim)
            im_dim = torch.FloatTensor(dim).repeat(1,2)                        
            if CUDA:
                im_dim = im_dim.cuda()
                img = img.cuda()          
            with torch.no_grad():   
                output = model(Variable(img), CUDA)
            #print(output)
            bar.update(frames)
            frames += 1
            output = write_results(output, confidence, num_classes, nms = True, nms_conf = nms_thresh)
            if type(output) == int:
                #frames += 1
                #print("FPS of the video is {:5.2f}".format( frames / (time.time() - start)))
                #cv2.imshow("frame", orig_im)
                key = cv2.waitKey(1)
                if key & 0xFF == ord('q'):
                    break
                continue
            if cap.get(cv2.CAP_PROP_POS_MSEC)>(timeCursor+60000):
                ti+=1
            timeCursor=cap.get(cv2.CAP_PROP_POS_MSEC)
            
            im_dim = im_dim.repeat(output.size(0), 1)
            scaling_factor = torch.min(inp_dim/im_dim,1)[0].view(-1,1)
            output[:,[1,3]] -= (inp_dim - scaling_factor*im_dim[:,0].view(-1,1))/2
            output[:,[2,4]] -= (inp_dim - scaling_factor*im_dim[:,1].view(-1,1))/2
            output[:,1:5] /= scaling_factor
            for i in range(output.shape[0]):
                output[i, [1,3]] = torch.clamp(output[i, [1,3]], 0.0, im_dim[i,0])
                output[i, [2,4]] = torch.clamp(output[i, [2,4]], 0.0, im_dim[i,1])
            classes = load_classes(classes_file)
            colors = pkl.load(open("pallete", "rb"))
            
            counter=frames+count_files
            image_file='frame'+str(counter)+'.jpg'
            cv2.imwrite(output_frames+os.path.sep+image_file, orig_im)
            list(map(lambda x: write(x, orig_im), output)) 
            cv2.imwrite(output_predictions+os.path.sep+image_file, orig_im)
            preds=list(map(lambda x: get_classes(x), output)) 

            persons=preds.count(2)
            result.loc[counter]=[video,persons,len(preds)-persons,timeCursor,cap.get(cv2.CAP_PROP_POS_FRAMES),ti]
            
            #cv2.imshow("frame", orig_im)
            key = cv2.waitKey(1)
            if key & 0xFF == ord('q'):
                break
            last_success_frame=frames
            
            
            #print("FPS of the video is {:5.2f}".format( frames / (time.time() - start)))
        else:
            break

/home/ds12/anaconda3/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")
  0% (6 of 22525) |                      | Elapsed Time: 0:00:00 ETA:   0:07:03

Analyze input/0000000000000000-180521-122443-123944-01p406000000.avi


  0% (5 of 22531) |                      | Elapsed Time: 0:00:00 ETA:   0:08:15

Analyze input/0000000000000000-180521-133943-135444-01p406000000.avi


  0% (5 of 22533) |                      | Elapsed Time: 0:00:00 ETA:   0:08:34

...

  0% (6 of 22525) |                      | Elapsed Time: 0:00:00 ETA:   0:07:14

Analyze input/0000000000000000-180521-120943-122443-01p406000000.avi


  0% (5 of 22540) |                      | Elapsed Time: 0:00:00 ETA:   0:08:52

Analyze input/0000000000000000-180521-112443-113944-01p406000000.avi


 99% (22534 of 22540) |################# | Elapsed Time: 0:06:50 ETA:   0:00:00

In [ ]:
from IPython.display import Image

In [ ]:
images=[]
for each in result.sample(n=5)[['countPerson','countBags']].iterrows():
    images.append(Image(filename=output_frames+os.path.sep+'frame'+str(int(each[0]))+'.jpg',retina=True))
    images.append(Image(filename=output_predictions+os.path.sep+'frame'+str(int(each[0]))+'.jpg',retina=True))
images

In [ ]:
images=[]
for each in result.sample(n=5)[['countPerson','countBags']].iterrows():
    images.append('frame'+str(int(each[0]))+'.jpg')
images

In [ ]:
for each in images:
    image_f = cv2.imread(output_frames+os.path.sep+each)
    image_p = cv2.imread(output_predictions+os.path.sep+each)
# I just resized the image to a quarter of its original size
    image_f = cv2.resize(image_f, (0, 0), None, .25, .25)
    image_p = cv2.resize(image_p, (0, 0), None, .25, .25)
    numpy_horizontal = np.hstack((image_f, image_p))
    numpy_horizontal_concat = np.concatenate((image_f, image_p), axis=1)

    cv2.imshow('Numpy Horizontal', numpy_horizontal_concat)

In [ ]:
cv2.imshow('Main', image)

In [ ]:
image = cv2.imread("kuju.jpg") #Load image
cv2.imshow('Main', image)
aa = cv2.imread("kuju.jpg",0) #Apply filter bb = cv2.imread("kuju.jpg",1)

cv2.imshow("frame1",aa) #display in windows cv2.imshow("frame2",bb)

cv2.waitKey(0) cv2.destroyAllWindows()**



In [ ]:
for each in result.sample(n=5)[['countPerson','countBags']].iterrows():
    display(Image(filename=output_frames+os.path.sep+'frame'+str(int(each[0]))+'.jpg',retina=True))
    display(Image(filename=output_predictions+os.path.sep+'frame'+str(int(each[0]))+'.jpg',retina=True))

In [ ]:
for each in result.timeInterval.unique():
    start=result[result['timeInterval']==each].time.min()
    finish=result[result['timeInterval']==each].time.max()
    clip = ffmpeg_extract_subclip(result[result['timeInterval']==each].file.iloc[0],start/1000,finish/1000,os.path.join(clipFolder,'clip'+str(each)+'.mp4'));